# set SYMPHONY_API_KEY as an environment variable before importing pysymphony

In [1]:
from pysymphony import SymphonyClient, settings
import logging

# set logging level to INFO for basic logs
logging.basicConfig(level=logging.INFO)

# add a local tool
# ensure to add a docstring for the tool description, and define the input parameter types
def add_tool(a: int, b: int):
    """
    Add two numbers
    """
    print(f"Adding {a} and {b}")
    return a + b

settings.add_local_tool(add_tool)

/Users/chinmayshrivastava/Documents/Github/pysymphony/venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "schema" in "LocalTool" shadows an attribute in parent "BaseModel"
  warnings.warn(
INFO:root:GET https://inference.composition-labs.com/inference/tools/list: RUNNING
INFO:root:GET https://inference.composition-labs.com/inference/tools/list: SUCCESS


In [2]:
# initialize the symphony client
client = SymphonyClient()

In [3]:
# check available tools
client.tools

[{'id': 'pplx_online',
  'name': 'Perplexity Online',
  'description': 'Useful for getting internet enabled (access to internet) natural language answers to input queries without citations.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'openai_img_to_text',
  'name': 'OpenAI Image to Text',
  'description': 'Useful for analyzing and describing images, detecting objects, and extracting text from images.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'clinical_trials_gov_data_retriever',
  'name': 'Clinical Trials Gov Data Retriever',
  'description': 'Useful for retrieving a list of clinical studies for the pharma and biotech indistries, based on a given query, or details of a single clinical study based on a given NCT number. Pass studies_query to get a list of studies based on the query. Pass nct_id to get details of a single study. Never pass both studies_query and nct_id.',
  'type': <ToolType.SYMPHONY: 'symphony'>},
 {'id': 'competitor_analysis_tool',
  'name': 'Comp

In [4]:
# generate a workflow

# 1. define relevant parameters
useful_tools = ["add_tool"]
parameters = {
    "max_steps": 1
}
task_description = "For the given list of numbers, add them all up"

# 2. generate the workflow
workflow = client.generate_workflow(task_description=task_description, tools=useful_tools, parameters=parameters)

# print the workflow
print(workflow.__dict__)

INFO:root:POST https://inference.composition-labs.com/inference/planner/generateworkflow: SUCCESS
INFO:root:GET https://inference.composition-labs.com/inference/planner/getworkflows: RUNNING
INFO:root:GET https://inference.composition-labs.com/inference/planner/getworkflows: SUCCESS


{'id': 'f6251236-bf34-4c0d-8b1e-ad5e45c20142', 'task_description': 'For the given list of numbers, add them all up', 'nodes': [{'id': 'c312170e-ed98-446e-b7c9-4b5fdd2b6829', 'step_index': 1, 'input': 'A list of numbers', 'job': 'Take the given list of numbers and calculate the sum of all the numbers in the list.', 'output': 'The total sum of the numbers in the list.', 'prompt': 'Calculate the sum of all the numbers in the given list and provide the total sum as a single numerical value.', 'dependencies': [], 'created_at': '2024-10-08T01:47:08.889845', 'updated_at': '2024-10-08T01:47:08.889845', 'tools': []}], 'runs': [], 'parameters': {}, 'session': <requests.sessions.Session object at 0x1087a9fa0>, 'tools': ['add_tool'], 'base_url': 'https://inference.composition-labs.com/inference'}


In [5]:
# run the workflow

# 1. initialize the run
run = workflow.init_run(input="12313123, 12312312324, 12313, 123123123")

INFO:root:POST https://inference.composition-labs.com/inference/runner/initrun: SUCCESS


In [6]:
run.run()

INFO:root:POST https://inference.composition-labs.com/inference/runner/runstep: SUCCESS


<Status.SUCCESS: 'success'>

In [7]:
run.ios

{'0': {'input': None, 'output': '12313123, 12312312324, 12313, 123123123'},
 '1': {'input': 'A list of numbers: 12313123, 12312312324, 12313, 123123123',
  'output': "To calculate the sum of the numbers in the list, we will add them together:\n\n1. 12313123\n2. 12312312324\n3. 12313\n4. 123123123\n\nNow, let's perform the addition step by step:\n\n- First, add the first two numbers:\n  12313123 + 12312312324 = 12324425447\n\n- Next, add the third number:\n  12324425447 + 12313 = 12324437760\n\n- Finally, add the last number:\n  12324437760 + 123123123 = 12324560883\n\nThus, the total sum of all the numbers in the list is **12324560883**."}}